In [4]:
import requests
from bs4 import BeautifulSoup


def scrapetle():
    import urllib2

    data = urllib2.urlopen("http://celestrak.com/NORAD/elements/stations.txt").read(168) # read only 168 chars
    lines = data.split("\n") # then split it into lines

    #for line in lines:
    #    print line
    return (lines)

def scrapetle2():
    url = 'http://celestrak.com/NORAD/elements/stations.txt'
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, "lxml")
    mytle=(soup.text[:168])
    lines = mytle.splitlines()
    return (lines)


def fiximplicitdecimals (tempvar):
    import numpy as np
    bstarint=int(tempvar[0:5])
    bstarexp=10**float(tempvar[5:7])
    absbstar=abs(bstarint)
    sgnbstar=np.sign(bstarint)
    strabsbstar="0."+str(absbstar)
    bstarfloat=sgnbstar*float(strabsbstar)*bstarexp
    return bstarfloat

def tletodict(lines):
    import numpy as np
    l1=lines[1].split()
    l2=lines[2].split()

    satellite_number=l1[1] 

    isstle={}

    tempvar =l1[3]
    # getting keplerian elements from first line of TLE
    isstle['satellite_number'] =l1[1]

    isstle['epoch_year'] =int('20'+tempvar[0:2])
    isstle['epoch']=float(tempvar[2:12])
    isstle['first_time_deriv_mean_motion']=float(l1[4])
    tempvar=l1[5]
    isstle['second_time_deriv_mean_motion']=fiximplicitdecimals(l1[5])
    isstle['bstar_drag']    = fiximplicitdecimals(l1[6])
    print  10**float(tempvar[6:8])  , isstle['bstar_drag'] 
    zeronum=l1[7]
    tempvar=l1[8]
    isstle['element_Set_no']=int(tempvar[0:3])
    # getting keplerian elements from second line of TLE
    isstle['inclination']    =float(l2[2])
    isstle['right_ascension_ascending_node']  =  float(l2[3])
    # todo: fix for when the values are negative with an assumed decimal point
    # todo: the appended -0
    isstle['eccentricity']    = float("0."+l2[4])
    isstle['argument_perigee']    =float(l2[5])
    isstle['mean_anomaly']    =float(l2[6])
    tempvar=lines[2]
    tempvar=tempvar[52:69]
    isstle['mean_motion']   =float(tempvar[0:11])
    #print ('tledict', tempvar)
    isstle['revolution_number_at_epoch'] =int(tempvar[11:16])
    return isstle


In [5]:
def AddTLEToDB():
    lines=scrapetle()

    isstle= tletodict(lines)

    import psycopg2
    import sys
    import numpy as np


    con = None

    try:

        con = psycopg2.connect(host='asdc.space.dtu.dk', database='gld', user='gmurphy', password='j-l%CHB2Za') 
        #con = psycopg2.connect(host='localhost', database='gld', user='gmurphy', password='j-l%CHB2Za') 
        #con = psycopg2.connect(host='localhost', database='gld', user='asdcadmin', password='ASIMrocks') 
        cur = con.cursor()
        cur.execute("SELECT * FROM isstle LIMIT 0")

        cur.execute("SET TIME ZONE 'UTC';")  # UTC 
        ##cur.execute("INSERT INTO isstle ( satellite_number,       epoch_year  ,epoch,                    first_time_deriv_mean_motion   ,second_time_deriv_mean_motion,bstar_drag    ,element_Set_no   ,inclination    ,right_ascension_ascending_node    ,eccentricity    ,argument_perigee    ,mean_anomaly    ,mean_motion   ,revolution_number_at_epoch    )         VALUES (%s, %s, %s, %s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s  )",          ( isstle['satellite_number']   ,  isstle['epoch_year']  ,isstle['epoch'], isstle['first_time_deriv_mean_motion']   ,isstle['second_time_deriv_mean_motion'],isstle['bstar_drag']    ,isstle['element_Set_no']   ,isstle['inclination']    ,isstle['right_ascension_ascending_node']    ,isstle['eccentricity']    ,isstle['argument_perigee']    ,isstle['mean_anomaly']    ,isstle['mean_motion']   ,isstle['revolution_number_at_epoch']           ))       
        ##cur.execute("INSERT INTO isstle ( satellite_number,       epoch_year  ,epoch,                    first_time_deriv_mean_motion   ,second_time_deriv_mean_motion,bstar_drag    ,element_Set_no   ,inclination    ,right_ascension_ascending_node    ,eccentricity    ,argument_perigee    ,mean_anomaly    ,mean_motion   ,revolution_number_at_epoch    )         VALUES (%s, %s, %s, %s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s  )",          (             1,1,1,1,1,1,1,1,1,1,1,1,1,1))       
        SQL = "INSERT INTO isstle (satellite_number, epoch_year, epoch,   first_time_deriv_mean_motion, second_time_deriv_mean_motion, bstar_drag,   element_Set_no, inclination,right_ascension_ascending_node,  eccentricity,argument_perigee,mean_anomaly,  mean_motion,revolution_number_at_epoch) VALUES (%s, %s, %s,  %s, %s, %s,     %s, %s, %s,   %s, %s, %s,   %s, %s );" # Note: no quotes
        data = (isstle['satellite_number']   ,  isstle['epoch_year']  ,isstle['epoch'], isstle['first_time_deriv_mean_motion']   ,isstle['second_time_deriv_mean_motion'],isstle['bstar_drag']    ,isstle['element_Set_no']   ,isstle['inclination']    ,isstle['right_ascension_ascending_node']    ,isstle['eccentricity']    ,isstle['argument_perigee']    ,isstle['mean_anomaly']    ,isstle['mean_motion']   ,isstle['revolution_number_at_epoch']  )
        cur.execute(SQL, data) # Note: no % operator
        
        #cur.execute("INSERT INTO isstle (satellite_number, epoch_year) VALUES (200,200);")
        

        cols = isstle.keys()
        vals = [isstle[x] for x in cols]
        vals_str_list = ["%s"] * len(vals)
        vals_str = ", ".join(vals_str_list)

        #cur.execute("INSERT INTO isstle ({cols}) VALUES ({vals_str})".format( cols = cols, vals_str = vals_str), vals)


        con.commit()
        print ('Successfully committed to database'     )

    except :
        print ('Error could not connect to database'     )
        #sys.exit(1)


    finally:

        if con:
            con.close()  
    import pprint
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(isstle)
    
AddTLEToDB()

1.0 4.7201e-05
Successfully committed to database
{   'argument_perigee': 41.2017,
    'bstar_drag': 4.7200999999999998e-05,
    'eccentricity': 0.0001107,
    'element_Set_no': 999,
    'epoch': 187.213397,
    'epoch_year': 2016,
    'first_time_deriv_mean_motion': 2.712e-05,
    'inclination': 51.6429,
    'mean_anomaly': 18.8026,
    'mean_motion': 15.54712277,
    'revolution_number_at_epoch': 779,
    'right_ascension_ascending_node': 328.1031,
    'satellite_number': '25544U',
    'second_time_deriv_mean_motion': 0.0}
